In [ ]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

In [1]:
import os
import pandas as pd
from datasets import load_dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline)
import torch
from peft import LoraConfig,PeftModel,prepare_model_for_kbit_training
from trl import SFTTrainer
from warnings import filterwarnings
filterwarnings("ignore")

2024-05-08 18:04:36.311480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 18:04:36.311539: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 18:04:36.312978: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Set Hugging Face Token:

In [44]:
os.environ["HF_TOKEN"]="************************************" # Enter Your Own Hugging Face Token Key

# Load Dataset:

In [3]:
dataset= load_dataset("garage-bAInd/Open-Platypus")
dataset

Generating train split:   0%|          | 0/24926 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source'],
        num_rows: 24926
    })
})

In [4]:
dataset["train"]

Dataset({
    features: ['input', 'output', 'instruction', 'data_source'],
    num_rows: 24926
})

In [5]:
dataset["train"].to_pandas()

,input,output,instruction,data_source
0,,To find the probability of the spinner landing...,A board game spinner is divided into three par...,MATH/PRM-800K
1,,"I need to choose 6 people out of 14, and the o...",My school's math club has 6 boys and 8 girls. ...,MATH/PRM-800K
2,,First we count the number of all 4-letter word...,How many 4-letter words with at least one cons...,MATH/PRM-800K
3,,She can do this if and only if at least one of...,Melinda will roll two standard six-sided dice ...,MATH/PRM-800K
4,,Think of the problem as a sequence of H's and ...,"Let $p$ be the probability that, in the proces...",MATH/PRM-800K
...,...,...,...,...
24921,,There are several formulas that generate all p...,Can we find a formula to generate all prime nu...,guanaco
24922,,"Sure, here are some of the best universities:\...",What are some of the best university's for stu...,guanaco
24923,,[Enter stage left]\n\nFirst Citizen: Good morr...,Write me a SPL (Shakespeare Programming Langua...,guanaco
24924,,That's a very thoughtful idea! Here are some s...,Hi. I want to make an embossed picture for my ...,guanaco


In [6]:
# Assuming dataset is your Dataset object
columns_to_remove = ['input', 'data_source']
dataset = dataset.remove_columns(columns_to_remove)

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['output', 'instruction'],
        num_rows: 24926
    })
})

In [9]:
def chat_template(example):
    example["instruction"] = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    return example

dataset= dataset.map(chat_template)

Map:   0%|          | 0/24926 [00:00<?, ? examples/s]

In [10]:
dataset["train"][0]

{'output': 'To find the probability of the spinner landing on $C$, I need to subtract the probabilities of the spinner landing on $A$ and $B$ from $1$, since the sum of the probabilities of all possible outcomes is $1$. I can write this as an equation: $P(C) = 1 - P(A) - P(B)$. I know that $P(A) = \\frac{1}{3}$ and $P(B) = \\frac{5}{12}$, so I can plug those values into the equation and simplify. I get: $P(C) = 1 - \\frac{1}{3} - \\frac{5}{12} = \\frac{12}{12} - \\frac{4}{12} - \\frac{5}{12} = \\frac{3}{12}$. I can reduce this fraction by dividing the numerator and denominator by $3$, and I get: $P(C) = \\frac{1}{4}$. ',
 'instruction': '### Instruction:\nA board game spinner is divided into three parts labeled $A$, $B$  and $C$. The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is the probability of the spinner landing on $C$? Express your answer as a common fraction.\n\n### Response:\n'}

In [11]:
base_model= "NousResearch/Llama-2-7b-chat-hf"
new_model= "llama-2-7b-platypus_by_shorya_sharma"
tokenizer= AutoTokenizer.from_pretrained(base_model,use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [12]:
dataset=dataset["train"]

In [13]:
dataset= dataset.select(range(1000))

In [14]:
dataset.to_pandas()


,output,instruction
0,To find the probability of the spinner landing...,### Instruction:\nA board game spinner is divi...
1,"I need to choose 6 people out of 14, and the o...",### Instruction:\nMy school's math club has 6 ...
2,First we count the number of all 4-letter word...,### Instruction:\nHow many 4-letter words with...
3,She can do this if and only if at least one of...,### Instruction:\nMelinda will roll two standa...
4,Think of the problem as a sequence of H's and ...,### Instruction:\nLet $p$ be the probability t...
...,...,...
995,"By Cauchy-Schwarz,\n\[(y^2 + x^2)(3x^2 + y^2) ...",### Instruction:\nLet $x$ and $y$ be positive ...
996,I notice that the problem involves the arithme...,"### Instruction:\nThe sequence $a_1,$ $a_2,$ $..."
997,"So, if we add $k$ to each of the numbers $36$,...",### Instruction:\nIf the integer $k^{}_{}$ is ...
998,"Let $f(x) = ax^4+bx^3+cx^2+bx+a$. Thus, the pr...",### Instruction:\nSuppose\n$$a(2+i)^4 + b(2+i)...


# QLoRA Config:

In [15]:
# Quantization Config 
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True)

# LoRA COnfig
peft_config= LoraConfig(lora_alpha=15,
                        lora_dropout=0.1,
                        bias="none",
                        task_type="CASUAL_LM")

# Loading Base Model:

In [16]:
model= AutoModelForCausalLM.from_pretrained(base_model,
                                            quantization_config=bnb_config,
                                            device_map={"":0})
model.config.use_cache=False
model.config.pretraining_tp=1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [17]:
model= prepare_model_for_kbit_training(model)

In [18]:
training_args=TrainingArguments(output_dir="/kaggle/working/results",
                               num_train_epochs=1,
                               per_device_train_batch_size=4,
                               gradient_accumulation_steps=1,
                               evaluation_strategy="steps",
                               eval_steps=1000,
                               logging_steps=25,
                               optim="paged_adamw_8bit",
                               learning_rate=2e-4,
                               lr_scheduler_type='linear',
                               warmup_steps=10,
                               report_to="tensorboard",
                               max_steps=-1)

trainer= SFTTrainer(model=model,
                   train_dataset=dataset,
                   eval_dataset=dataset,
                   peft_config=peft_config,
                   dataset_text_field="instruction",
                   max_seq_length=512,
                   tokenizer=tokenizer,
                   args=training_args)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=250, training_loss=1.301362075805664, metrics={'train_runtime': 1161.9066, 'train_samples_per_second': 0.861, 'train_steps_per_second': 0.215, 'total_flos': 6367228370780160.0, 'train_loss': 1.301362075805664, 'epoch': 1.0})

In [20]:
trainer.model.save_pretrained(new_model)

# Model Inferencing:

In [27]:
prompt= "can you suggest me a method to optimize my machine learning model?"
pipe= pipeline(task='text-generation',model=model,tokenizer=tokenizer,max_length=200)
result= pipe(f"### Instruction:\n{prompt}\n\n### Response:\n")
print(result[0]['generated_text'])

### Instruction:
can you suggest me a method to optimize my machine learning model?

### Response:

There are many ways to optimize a machine learning model, depending on the specific problem and the type of model being used. Here are a few common methods:

1. **Regularization**: This involves adding a penalty term to the loss function to prevent overfitting. L1 and L2 regularization are common techniques used to reduce the magnitude of model weights and prevent overfitting.
2. **Early Stopping**: This involves monitoring the validation loss during training and stopping the training process when the validation loss stops improving. This can help prevent overfitting by stopping the training process before the model has a chance to fit the noise in the training data.
3. **Batch Normalization**: This involves normalizing the inputs to each layer of the model, which can help improve the stability and convergence of the training process


# Empty VRAM:

In [55]:
# del model
# del pipe
# del trainer
# import gc 
# gc.collect()

# Merge Base Model With Trained Adapters:

In [40]:
# Reload model in FP16 and merge it with LoRA Weights
model= AutoModelForCausalLM.from_pretrained(base_model,
                            low_cpu_mem_usage=True,
                            return_dict=True,
                            torch_dtype=torch.float16,
                            device_map={"":0})

# Reload base model and load the QLoRA Adapters
model= PeftModel.from_pretrained(model,new_model)
model= model.merge_and_unload()

# Reload tokenizer to save it 
tokenizer= AutoTokenizer.from_pretrained(base_model,trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Push the Fine Tuned Model and Tokenizer to the Hugging Face Hub:

In [41]:
import locale
locale.getpreferredencoding=lambda :"UTF-8"

In [46]:
model.push_to_hub("Shorya22/llama-2-7b-platypus_by_shorya_sharma")
tokenizer.push_to_hub("Shorya22/llama-2-7b-platypus_by_shorya_sharma")

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shorya22/llama-2-7b-platypus_by_shorya_sharma/commit/fdc16edb3cdd1e75048a3eac0ddc3255de1c88d8', commit_message='Upload tokenizer', commit_description='', oid='fdc16edb3cdd1e75048a3eac0ddc3255de1c88d8', pr_url=None, pr_revision=None, pr_num=None)